In [8]:
!pip3 install tensorflow
!pip3 install keras

     |████████████████████████████████| 195.7 MB 33 kB/s  eta 0:00:01    |████████████████▎               | 99.4 MB 1.6 MB/s eta 0:01:02     |████████████████▊               | 102.4 MB 384 kB/s eta 0:04:03     |██████████████████              | 109.7 MB 2.1 MB/s eta 0:00:41     |███████████████████             | 115.6 MB 2.5 MB/s eta 0:00:32     |███████████████████▌            | 119.0 MB 3.3 MB/s eta 0:00:23     |███████████████████▌            | 119.2 MB 3.3 MB/s eta 0:00:23     |████████████████████▏           | 123.3 MB 2.6 MB/s eta 0:00:29     |██████████████████████▋         | 138.1 MB 2.7 MB/s eta 0:00:22     |███████████████████████▏        | 141.9 MB 2.1 MB/s eta 0:00:27     |██████████████████████████▏     | 159.9 MB 2.8 MB/s eta 0:00:13     |███████████████████████████     | 165.1 MB 2.9 MB/s eta 0:00:11     |█████████████████████████████▊  | 181.6 MB 2.8 MB/s eta 0:00:05��████████████▊| 194.0 MB 1.8 MB/s eta 0:00:01
     |████████████████████████████████| 129 kB 2.5 MB/s et

In [3]:
### import libraries
import keras
from keras_preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras import regularizers, optimizers
from keras.models import Sequential

from keras.applications.densenet import DenseNet121
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
from keras.models import load_model
from keras.utils.vis_utils import plot_model

import pandas as pd
import numpy as np
from pathlib import Path

import matplotlib
matplotlib.use("Agg") # set the matplotlib backend so figures can be saved in the background
 
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt


In [4]:
dtrain=pd.read_csv("CheXpert-v1.0-small/train.csv", nrows=100)
dtrain = dtrain.fillna(0)
# import valid data info from valid.csv
dnew=pd.read_csv("CheXpert-v1.0-small/valid.csv", nrows=50)
dnew = dnew.fillna(0)

#add dnew to dtrain to re-split since valid data in data set is very small
dtrain = dtrain.append(dnew)

#pre-process data: remove Lateral images
dtrain = dtrain[~dtrain[dtrain.columns[3]].str.contains("Lateral")]
#pre-process data: drop selected features - only images as inputs
dtrain = dtrain.drop(["Sex", "Age", "Frontal/Lateral", "AP/PA"], axis=1)
print(dtrain.shape)
dtrain.describe().transpose()

(110, 15)


,count,mean,std,min,25%,50%,75%,max
No Finding,110.0,0.181818,0.387460,0.0,0.0,0.0,0.0,1.0
Enlarged Cardiomediastinum,110.0,0.109091,0.456260,-1.0,0.0,0.0,0.0,1.0
Cardiomegaly,110.0,0.109091,0.391315,-1.0,0.0,0.0,0.0,1.0
Lung Opacity,110.0,0.463636,0.553177,-1.0,0.0,0.0,1.0,1.0
Lung Lesion,110.0,0.045455,0.209252,0.0,0.0,0.0,0.0,1.0
Edema,110.0,0.118182,0.351454,-1.0,0.0,0.0,0.0,1.0
Consolidation,110.0,-0.009091,0.370852,-1.0,0.0,0.0,0.0,1.0
Pneumonia,110.0,0.000000,0.191565,-1.0,0.0,0.0,0.0,1.0
Atelectasis,110.0,0.154545,0.544975,-1.0,0.0,0.0,0.0,1.0
Pneumothorax,110.0,0.109091,0.367123,-1.0,0.0,0.0,0.0,1.0


In [5]:
# dealing with uncertanty (-1) values
dtrain = dtrain.replace(-1,1)
dtrain.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
No Finding,110.0,0.181818,0.387460,0.0,0.0,0.0,0.0,1.0
Enlarged Cardiomediastinum,110.0,0.218182,0.414902,0.0,0.0,0.0,0.0,1.0
Cardiomegaly,110.0,0.163636,0.371638,0.0,0.0,0.0,0.0,1.0
Lung Opacity,110.0,0.518182,0.501956,0.0,0.0,1.0,1.0,1.0
Lung Lesion,110.0,0.045455,0.209252,0.0,0.0,0.0,0.0,1.0
Edema,110.0,0.136364,0.344745,0.0,0.0,0.0,0.0,1.0
Consolidation,110.0,0.136364,0.344745,0.0,0.0,0.0,0.0,1.0
Pneumonia,110.0,0.036364,0.188050,0.0,0.0,0.0,0.0,1.0
Atelectasis,110.0,0.318182,0.467902,0.0,0.0,0.0,1.0,1.0
Pneumothorax,110.0,0.145455,0.354172,0.0,0.0,0.0,0.0,1.0


In [6]:
### split data into train/valid/test
dtrain = dtrain.sample(frac=1)
dvalid_size = round(0.1*dtrain.shape[0])
dtest_size = dvalid_size
dtr = dtrain[0:dtrain.shape[0]-dvalid_size-dtest_size+1]
dv = dtrain[dtrain.shape[0]-dvalid_size-dtest_size:dtrain.shape[0]-dvalid_size+1]
dte = dtrain[dtrain.shape[0]-dvalid_size:dtrain.shape[0]+1]

print(dv.shape)

(12, 15)


In [7]:
### data generation for Keras 
train_datagen=ImageDataGenerator(rescale=1./255)
test_datagen=ImageDataGenerator(rescale=1./255.)
valid_datagen=ImageDataGenerator(rescale=1./255.)

target_size = (224,224)
#target_size = (299,299)
#target_size = (75,75)
train_generator=train_datagen.flow_from_dataframe(dataframe=dtr, directory=None , x_col="Path", y_col=list(dtr.columns[1:15]), class_mode="other", target_size=target_size, batch_size=32)
valid_generator=valid_datagen.flow_from_dataframe(dataframe=dv, directory=None , x_col="Path", y_col=list(dv.columns[1:15]), class_mode="other", target_size=target_size, batch_size=32)
test_generator=test_datagen.flow_from_dataframe(dataframe=dte, directory=None , x_col="Path", y_col=list(dte.columns[1:15]), class_mode="other", target_size=target_size, shuffle = False, batch_size=1)

Found 89 validated image filenames.
Found 12 validated image filenames.
Found 11 validated image filenames.


In [8]:
### model architecture design/selection
# create the base pre-trained model
base_model = DenseNet121(include_top = False, weights='imagenet')

# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
# and a logistic layer
predictions = Dense(14, activation='sigmoid')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional DenseNet layers
for layer in base_model.layers:
    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)

#model training
adam = keras.optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
model.compile(optimizer= adam, loss='binary_crossentropy', metrics=['accuracy'])

#plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)
#print(model.summary())

29097984/29084464 [==============================] - 12s 0us/step


/usr/local/lib/python3.9/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(
